In [118]:
# Install dependencies (run once per environment)
%pip install -q dspy python-dotenv google-genai fal-client pillow ffmpeg-python
print("Note: restart the kernel if packages were freshly installed.")


Note: you may need to restart the kernel to use updated packages.
Note: restart the kernel if packages were freshly installed.


In [119]:
# Basic imports and environment setup
import os
import json
import dspy
from dotenv import load_dotenv
from dspy import History

load_dotenv()

# Configure LM (Anthropic or OpenAI OK; using OpenAI mini for text-only here)
lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000)
dspy.configure(lm=lm)

print("DSPy configured for Video Response Generator DAG.")


DSPy configured for Video Response Generator DAG.


In [120]:
# Pydantic models equivalent to LangGraph `models.py`
from typing import Dict
from pydantic import BaseModel, Field

class BuyingSituation(BaseModel):
    situation: str
    location: str
    trigger: str
    evaluation: str
    conclusion: str

class BuyingSituationsOutput(BaseModel):
    persona: str = Field(..., description="Name or identifier for the persona")
    buying_situations: Dict[str, BuyingSituation]

class SceneDescription(BaseModel):
    persona: str
    appearance: str

class PromptModel(BaseModel):
    scene_description: SceneDescription
    quote: str
    prompt_string: str

class PromptOutput(BaseModel):
    prompt: PromptModel


In [121]:
# Core tools ported from LangGraph DAG (plain Python functions)
import time
import uuid
from pathlib import Path
from typing import Any, List, Dict, Optional, cast

import httpx
import fal_client

# Phase 0: init and persona catalog

def init_and_create_directory() -> Dict[str, str]:
    execution_id = uuid.uuid4().hex
    work_dir = Path(f"/tmp/n8n/{execution_id}")
    work_dir.mkdir(parents=True, exist_ok=True)
    for f in ["videos.txt", "final_output.mp4"]:
        try:
            (work_dir / f).unlink(missing_ok=True)
        except Exception:
            pass
    return {"execution_id": execution_id, "work_dir": str(work_dir)}


def define_personas() -> Dict[str, Dict[str, Dict[str, str]]]:
    personas: Dict[str, Dict[str, str]] = {
        "Omar US Developer": {
            "name": "Omar Ali",
            "age": "32",
            "gender": "Male",
            "location": "San Francisco",
            "occupation": "Software Engineer",
            "income": "$200,000",
            "background": (
                "This individual's life revolves around exploration and understanding. They are deeply curious about the world, people, and how things work, often delving into complex topics such as philosophy, history, and the human psyche. Their online presence is a reflection of their inner world - an active, thoughtful space where they seek to make connections, exchange ideas, and learn from others. They love the act of learning and self-improvement. They have a strong desire to make things better for themselves and the world around them, expressed through their writing and interactions. This person is committed to their craft, valuing beauty and truth, and finds joy in the small, everyday aspects of life. They are fascinated by the power of human connection and the potential for growth within communities. They are known for their inquisitive nature, often posing questions and seeking different perspectives to broaden their understanding of the world. They have an open mind and are constantly seeking a deeper understanding of complex concepts and human behavior. Their curiosity is a constant source of inspiration, driving them to explore new ideas, challenge their beliefs, and make meaningful connections with others."
            ),
        },
        "Sarah UK Nurse": {
            "name": "Sarah Thomspon",
            "age": "55",
            "gender": "Woman",
            "location": "Manchester, UK",
            "occupation": "Nurse",
            "income": "GBP 55k",
            "background": (
                "Sarah is a dedicated Registered Nurse with over thirty years of experience in the NHS. Raised in a working-class family, she was inspired to pursue nursing by her strong desire to help others and provide compassionate care. She is married, owns her home, and is a devout Christian, finding strength in her faith and family. Sarah is a strong advocate for her patients and believes healthcare is a fundamental right. She enjoys reading, gardening, and spending time outdoors, which helps her manage the stress that comes with her demanding job. Throughout her career, she has consistently strived to improve her skills and provide the best possible care to her patients."
            ),
        },
        "Emily US Foodie": {
            "name": "Emily Carter",
            "age": "21",
            "gender": "Female",
            "location": "Los Angeles, CA",
            "occupation": "Student",
            "income": "NA",
            "background": (
                "Emily Carter is a 21-year-old college student at UCLA, majoring in Digital Marketing. Originally from a small town in Oregon, she moved to Los Angeles to pursue her passion for marketing and content creation. She's an avid foodie, constantly exploring LA's diverse culinary landscape and documenting her experiences on TikTok and Instagram. Her content focuses on restaurant reviews, food trends, and lifestyle content, and she hopes to work in social media marketing after graduation."
            ),
        },
        "Clara US Health Coach": {
            "name": "Clara Johnson",
            "age": "35",
            "gender": "Female",
            "location": "New York City, NY",
            "occupation": "Health Coach",
            "income": "$70k",
            "background": (
                "Clara, a certified holistic health coach, was raised in a culturally diverse household where natural wellness was a way of life. Inspired by her family's practices, she pursued a certification in holistic health and launched her coaching business in NYC. Through her work at NaturalHealth Inc., Clara focuses on promoting holistic beauty methods, integrating wellness with skincare, and educating others on the benefits of natural products. She is passionate about community workshops and uses her platform to share DIY beauty techniques, encouraging others to embrace natural wellness. She is a Buddhist who believes beauty comes from within and enjoys meditation, nature walks, and creating healthy meals."
            ),
        },
        "Jordan US Tattoo Artist": {
            "name": "Jordan McCulloch",
            "age": "Age 5",
            "gender": "Male",
            "location": "Portland, OR, USA",
            "occupation": "Tattoo Artist",
            "income": "$48k",
            "background": (
                "Jordan grew up in a small coastal town in Oregon before moving to Portland to embrace the city’s alternative scene and vibrant art community. With a talent for visual storytelling and a welcoming demeanor, Jordan found a calling in tattoo artistry, channeling creativity into meaningful body art for friends and clients. Passionate about authentic self-expression and forging real connections, Jordan has built a loyal client base and a network of close friends who share a love of indie music, art shows, and late-night philosophical discussions. Jordan values openness in relationships and tends to express themselves candidly, both in art and conversation. When not at the tattoo studio, Jordan spends time volunteering for LGBTQ+ causes, making zines, and exploring the city’s eclectic food scene."
            ),
        },
    }
    return {"personas": personas}


def set_persona(personas: Dict[str, Dict[str, str]], persona_selection: str) -> Dict[str, str]:
    if not persona_selection or persona_selection not in personas:
        raise ValueError("persona_selection is missing or invalid")
    return personas[persona_selection]


# Phase 1: Buying situations via LM

def _get_llm_predict():
    return dspy.Predict("question -> answer")


def generate_buying_situations(persona: Dict[str, str]) -> Dict[str, Any]:
    if not persona:
        raise KeyError("persona missing; ensure set_persona ran first")
    system = (
        "Role play as {name}, a {age} {gender} from {location}, works as a {occupation} "
        "earning {income}. {background}. You have been invited to review a new product "
        "concept and reveal deep, personal insights into when you would buy this offer. "
        "Return only valid JSON with fields persona (string) and buying_situations (object with exactly 3 keys)."
    ).format(**persona)
    example = '{"persona":"Rhys UK Entrepreneur","buying_situations":{"celebrating_milestone":{"situation":"Celebrating a business milestone","location":"Private members\' club in London","trigger":"Closed a major deal","evaluation":"Considered champagne and cocktails, but wanted tradition","conclusion":"Chose premium whisky"},"hosting_partners":{"situation":"Hosting international partners","location":"Home office in Manchester","trigger":"Inviting overseas investors","evaluation":"Compared various spirits; whisky is distinctly British","conclusion":"Bought respected Scottish whisky"},"relaxing_weekend":{"situation":"Relaxing after a long week","location":"Apartment balcony overlooking the city","trigger":"Needed a way to unwind","evaluation":"Looked at beer, gin, wine; whisky appealed","conclusion":"Chose whisky as a ritual drink"}}}'
    prompt = (
        "Respond only with raw JSON (no markdown). Use this schema and ensure 3 buying_situations. Example: "
        f"{example}"
    )
    predictor = _get_llm_predict()
    raw = predictor(question=f"{system}\n\n{prompt}")
    text = getattr(raw, "answer", "")
    data = None
    for _ in range(2):
        try:
            data = json.loads(text)
            break
        except Exception:
            repair = predictor(question="Repair to valid JSON with the same schema: " + text)
            text = getattr(repair, "answer", "")
    if data is None:
        raise ValueError("Failed to parse buying situations JSON")
    validated = BuyingSituationsOutput.model_validate(data)
    situations_list = list(validated.buying_situations.values())
    return {
        "buying_situations": validated.buying_situations,
        "situations_list": situations_list,
        "situation_index": 0,
        "prompts_json": [],
        "prompt_strings": [],
    }


def generate_prompt_for_current(persona: Dict[str, str], situations_list: List[BuyingSituation], situation_index: int) -> Dict[str, Any]:
    if not (0 <= situation_index < len(situations_list)):
        raise IndexError("situation_index out of range")
    situation = situations_list[situation_index]
    template = (
        "You are a director of a qualitative research agency that specialises in creating realistic simulated testimonials that capture buying situations for new product concepts.\n\n"
        "Your tasks are to: 1) Craft persona description, 2) scene appearance, 3) a short 6–8s quote, 4) assemble final prompt string.\n"
        "Return a structured object strictly matching this Pydantic schema: PromptOutput(prompt: PromptModel(scene_description: SceneDescription(persona, appearance), quote, prompt_string)). No markdown.\n"
        "Important: scene_description.persona MUST be a single short string with the persona's name only (e.g., 'Omar Ali'). Do NOT return an object for this field.\n"
        "Return exactly these keys: {\"prompt\": {\"scene_description\": {\"persona\": \"...\", \"appearance\": \"...\"}, \"quote\": \"...\", \"prompt_string\": \"...\"}}.\n\n"
        "Persona\n"
        f"Name: {persona['name']}\n"
        f"Age: {persona['age']}\n"
        f"Gender: {persona['gender']}\n"
        f"Location: {persona['location']}\n"
        f"Occupation: {persona['occupation']}\n"
        f"Income: {persona['income']}\n"
        f"Background: {persona['background']}\n\n"
        "Buying Situation\n"
        f"Situation: {situation.situation}\n"
        f"Location: {situation.location}\n"
        f"Trigger: {situation.trigger}\n"
        f"Evaluation: {situation.evaluation}\n"
        f"Conclusion: {situation.conclusion}"
    )
    predictor = dspy.Predict("instruction -> json_text")
    raw = predictor(instruction=template)
    text = getattr(raw, "json_text", "")
    data = None
    for _ in range(2):
        try:
            data = json.loads(text)
            break
        except Exception:
            repair = dspy.Predict("text -> json_text")(text="Repair to valid JSON only with the same schema. Ensure scene_description.persona is a single string (name only), not an object. Return exactly {\"prompt\": {\"scene_description\": {\"persona\": \"...\", \"appearance\": \"...\"}, \"quote\": \"...\", \"prompt_string\": \"...\"}}.\n\n" + text)
            text = getattr(repair, "json_text", "")
    if data is None:
        raise ValueError("Failed to parse prompt JSON")
    validated = PromptOutput.model_validate(data)
    return {"prompt_json": validated, "prompt_string": validated.prompt.prompt_string}


# Phase 3: FAL queue and video merging

def _fal_headers() -> Dict[str, str]:
    name = os.getenv("FAL_AUTH_HEADER_NAME", "Authorization")
    value = os.getenv("FAL_AUTH_HEADER_VALUE")
    if not value:
        api_key = os.getenv("FAL_API_KEY") or os.getenv("FAL_KEY")
        if api_key:
            value = f"Key {api_key}"
    return {name: value} if value else {}


def _normalize_fal_urls(status_url: Optional[str], response_url: Optional[str]) -> Dict[str, str]:
    def base_from(url: str) -> str:
        u = url.rstrip("/")
        if u.endswith("/status"):
            return u[:-7]
        if u.endswith("/response"):
            return u[:-9]
        # if already like /requests/{id}, return as base
        return u
    base = None
    if status_url:
        base = base_from(status_url)
    elif response_url:
        base = base_from(response_url)
    else:
        return {"status_url": status_url or "", "response_url": response_url or ""}
    return {"status_url": f"{base}/status", "response_url": f"{base}/response"}


def _extract_video_url_from_data(data: Dict[str, Any]) -> Optional[str]:
    if not isinstance(data, dict):
        return None
    if isinstance(data.get("video"), dict) and data.get("video", {}).get("url"):
        return str(data["video"]["url"])
    if data.get("video_url"):
        return str(data["video_url"])  # type: ignore
    if data.get("url") and str(data.get("url")).endswith(".mp4"):
        return str(data["url"])  # type: ignore
    return None


def _collect_final_response(client: httpx.Client, headers: Dict[str, str], base_url: str, wait_seconds: int = 1, max_polls: int = 60) -> Dict[str, Any]:
    urls = _normalize_fal_urls(status_url=None, response_url=base_url)
    print(f"[FAL] collect child: status={urls['status_url']} response={urls['response_url']}")
    # Poll child until completed
    last_status = None
    for attempt in range(max_polls):
        try:
            st = client.get(urls["status_url"], headers=headers)
            st.raise_for_status()
        except httpx.HTTPStatusError as ex:
            body = ex.response.text if ex.response is not None else ""
            code = ex.response.status_code if ex.response is not None else "?"
            print(f"[FAL] collect poll HTTP {code}; body={body[:300]}")
            raise
        sdata = st.json()
        s = str(sdata.get("status", "")).upper()
        if s != last_status:
            print(f"[FAL] collect poll attempt {attempt+1}/{max_polls}: status={s}")
            last_status = s
        if s == "COMPLETED":
            break
        if s in ("FAILED", "CANCELLED", "ERROR"):
            raise RuntimeError(f"FAL child job failed: {sdata}")
        time.sleep(max(0, int(wait_seconds)))
    else:
        raise TimeoutError(f"FAL child job did not complete after {max_polls} polls")
    # Fetch child response (prefer POST; some queue endpoints require POST to /response)
    try:
        resp = client.post(urls["response_url"], headers=headers, json={})
        if resp.status_code == 405:  # Method Not Allowed fallback
            resp = client.get(urls["response_url"], headers=headers)
        resp.raise_for_status()
    except httpx.HTTPStatusError as ex:
        body = ex.response.text if ex.response is not None else ""
        code = ex.response.status_code if ex.response is not None else "?"
        allow = ex.response.headers.get("Allow") if ex.response is not None else None
        print(f"[FAL] collect fetch HTTP {code}; allow={allow}; body={body[:300]}")
        raise
    return resp.json()


def resolve_video_url_follow_chain(client: httpx.Client, headers: Dict[str, str], base_url: str, wait_seconds: int = 1, max_polls: int = 60, max_chain: int = 6) -> Optional[str]:
    """Follow nested FAL response chains until a terminal payload exposes a video URL.
    Starts from a queue request base or its /response URL and iterates up to max_chain.
    """
    current = base_url
    last_keys: Optional[List[str]] = None
    for depth in range(max_chain):
        # Ensure the current request (root or child) is completed and fetch its response payload
        payload = _collect_final_response(
            client=client,
            headers=headers,
            base_url=current,
            wait_seconds=wait_seconds,
            max_polls=max_polls,
        )
        last_keys = list(payload.keys()) if isinstance(payload, dict) else None
        print(f"[FAL] chain depth {depth} keys={last_keys}")

        # Try to extract a direct video URL
        url_candidate = _extract_video_url_from_data(payload)
        if url_candidate:
            print(f"[FAL] chain depth {depth} resolved video URL: {url_candidate}")
            return url_candidate

        # Otherwise, if another response_url is provided, follow it on the next loop
        next_url = None
        if isinstance(payload, dict):
            next_url = payload.get("response_url") or payload.get("response")
        if next_url:
            print(f"[FAL] chain depth {depth} following nested response_url")
            current = str(next_url)
            continue

        # No further link to follow and no URL found
        print(f"[FAL] chain depth {depth} no video URL and no further response_url to follow")
        break

    return None


def submit_fal_requests(prompt_strings: List[str]) -> List[Dict[str, str]]:
    route = os.getenv("FAL_MODEL_ROUTE", "fal-ai/veo3")
    print(f"[FAL] route={route}; using fal_client")
    requests_info: List[Dict[str, str]] = []
    for idx, prompt in enumerate(prompt_strings):
        try:
            handler = fal_client.submit(route, arguments={"prompt": prompt})
        except Exception as ex:
            print(f"[FAL] submit({idx}) error: {ex}")
            raise
        request_id = getattr(handler, "request_id", None)
        if request_id is None and isinstance(handler, dict):
            request_id = handler.get("request_id")
        if not request_id:
            print(f"[FAL] submit({idx}) unexpected handler={handler}")
            raise ValueError("fal_client.submit did not return a request_id")
        print(f"[FAL] submit({idx}) ok: id={request_id}")
        requests_info.append({"request_id": str(request_id)})
    return requests_info


def poll_statuses(fal_requests: List[Dict[str, str]]) -> Dict[str, Any]:
    route = os.getenv("FAL_MODEL_ROUTE", "fal-ai/veo3")
    statuses: List[str] = []
    for idx, req in enumerate(fal_requests):
        request_id = req.get("request_id")
        if not request_id:
            raise ValueError(f"Missing request_id in request: {req}")
        print(f"[FAL] poll({idx}) request_id={request_id}")
        try:
            st = fal_client.status(route, request_id, with_logs=False)
        except Exception as ex:
            print(f"[FAL] poll({idx}) error: {ex}")
            raise
        # fal_client.status returns an object with .status or a dict with 'status'
        status_value = getattr(st, "status", None)
        if status_value is None and isinstance(st, dict):
            status_value = st.get("status")
        status_value = str(status_value or "").upper()
        print(f"[FAL] poll({idx}) status={status_value}")
        statuses.append(status_value)
    all_complete = all(s == "COMPLETED" for s in statuses) if statuses else False
    return {"statuses": statuses, "all_complete": all_complete}


def fetch_video_urls(fal_requests: List[Dict[str, str]] , wait_seconds: int = 1, max_polls: int = 60) -> List[str]:
    route = os.getenv("FAL_MODEL_ROUTE", "fal-ai/veo3")
    video_urls: List[str] = []
    for idx, req in enumerate(fal_requests):
        request_id = req.get("request_id")
        if not request_id:
            raise ValueError(f"Missing request_id in request: {req}")
        print(f"[FAL] fetch({idx}) request_id={request_id}")
        # Poll until completed (strict), mirroring previous behavior
        last_status = None
        for attempt in range(max_polls):
            st = fal_client.status(route, request_id, with_logs=False)
            status_value = getattr(st, "status", None)
            if status_value is None and isinstance(st, dict):
                status_value = st.get("status")
            s = str(status_value or "").upper()
            if s != last_status:
                print(f"[FAL] fetch({idx}) poll {attempt+1}/{max_polls} status={s}")
                last_status = s
            if s == "COMPLETED":
                break
            if s in ("FAILED", "CANCELLED", "ERROR"):
                raise RuntimeError(f"FAL job failed: {st}")
            time.sleep(max(0, int(wait_seconds)))
        else:
            raise TimeoutError(f"FAL job did not complete after {max_polls} polls: {request_id}")

        # Get the result and extract the video URL
        res = fal_client.result(route, request_id)
        url_candidate = _extract_video_url_from_data(res)
        print(f"[FAL] fetch({idx}) result keys={list(res.keys()) if isinstance(res, dict) else type(res)}")
        if not url_candidate:
            raise ValueError(f"No video URL in result: {res}")
        video_urls.append(url_candidate)
    return video_urls


In [122]:
# FAL subscribe-based generation helper (preferred)
from typing import Optional


def generate_videos_via_subscribe(prompt_strings: List[str], with_logs: bool = True) -> List[str]:
    route = os.getenv("FAL_MODEL_ROUTE", "fal-ai/veo3")
    print(f"[FAL] using subscribe flow; route={route}")

    def on_queue_update(update):
        if isinstance(update, fal_client.InProgress):
            if getattr(update, "logs", None):
                for log in update.logs:
                    try:
                        msg = log.get("message") if isinstance(log, dict) else str(log)
                        if msg:
                            print(msg)
                    except Exception:
                        pass

    video_urls: List[str] = []
    for idx, prompt in enumerate(prompt_strings):
        print(f"[FAL] subscribe({idx})")
        result = fal_client.subscribe(
            route,
            arguments={"prompt": prompt},
            with_logs=with_logs,
            on_queue_update=on_queue_update,
        )
        url_candidate = _extract_video_url_from_data(result)
        if not url_candidate:
            raise ValueError(f"No video URL in subscribe result: {result}")
        video_urls.append(url_candidate)
    return video_urls



In [123]:
# Download + ffmpeg helpers
import shutil
import subprocess

def download_videos(video_urls: List[str], work_dir: str) -> List[str]:
    Path(work_dir).mkdir(parents=True, exist_ok=True)
    files: List[str] = []
    with httpx.Client(timeout=None, follow_redirects=True) as client:
        for i, url in enumerate(video_urls):
            file_name = f"{i:02d}.mp4"
            file_path = Path(work_dir) / file_name
            with client.stream("GET", url) as resp:
                resp.raise_for_status()
                with open(file_path, "wb") as f:
                    for chunk in resp.iter_bytes():
                        if chunk:
                            f.write(chunk)
            files.append(str(file_path))
    return files


def prepare_concat_file(work_dir: str) -> None:
    list_path = Path(work_dir) / "videos.txt"
    lines = []
    for p in sorted(Path(work_dir).glob("*.mp4")):
        lines.append(f"file '{p.name}'\n")
    list_path.write_text("".join(lines), encoding="utf-8")


def _which(cmd: str) -> Optional[str]:
    return shutil.which(cmd)


def merge_videos_ffmpeg(work_dir: str) -> str:
    if not _which("ffmpeg"):
        raise RuntimeError("ffmpeg not found in PATH. Please install ffmpeg.")
    videos_txt = Path(work_dir) / "videos.txt"
    if not videos_txt.exists():
        raise FileNotFoundError(f"Missing {videos_txt}")
    final_path = Path(work_dir) / "final_output.mp4"
    cmd = [
        "ffmpeg", "-f", "concat", "-safe", "0", "-i", str(videos_txt), "-c", "copy", "-y", str(final_path)
    ]
    subprocess.run(cmd, cwd=str(work_dir), check=True, capture_output=True)
    return str(final_path)


def finalize_result(execution_id: str) -> Dict[str, str]:
    video_url = f"http://localhost:3001/video/{execution_id}/final_output.mp4"
    return {
        "videoUrl": video_url,
        "executionId": execution_id,
        "message": "Video processing complete",
    }


In [124]:
# Sequential orchestrator that mirrors the LangGraph DAG happy path

def run_video_response_agent(persona_selection: str, wait_seconds: int = 3, max_polls: int = 100) -> Dict[str, Any]:
    # Phase 0: init + personas
    init = init_and_create_directory()
    execution_id = init["execution_id"]
    work_dir = init["work_dir"]
    personas = define_personas()["personas"]
    persona = set_persona(personas, persona_selection)

    # Phase 1: generate buying situations
    bs = generate_buying_situations(persona)
    situations_list: List[BuyingSituation] = bs["situations_list"]

    # Phase 2: loop prompts for each situation
    prompts_json: List[PromptOutput] = []
    prompt_strings: List[str] = []
    for idx in range(len(situations_list)):
        out = generate_prompt_for_current(persona, situations_list, idx)
        pj = cast(PromptOutput, out["prompt_json"])  # type: ignore
        prompts_json.append(pj)
        prompt_strings.append(str(out["prompt_string"]))

    # Phase 3: FAL video generation via subscribe (streaming logs, waits for completion)
    video_urls = generate_videos_via_subscribe(prompt_strings, with_logs=True)
    print(f"[FAL] fetched video URLs: {video_urls}")

    # Download and merge
    _ = download_videos(video_urls, work_dir)
    prepare_concat_file(work_dir)
    final_path = merge_videos_ffmpeg(work_dir)

    # Final payload
    result = finalize_result(execution_id)
    return {
        "execution_id": execution_id,
        "work_dir": work_dir,
        "prompts": [p.model_dump() for p in prompts_json],
        "video_urls": video_urls,
        "final_path": final_path,
        "result": result,
    }


In [125]:
# Example usage (smoke test)
try:
    demo = run_video_response_agent(persona_selection="Omar US Developer", wait_seconds=1)
    print({
        "execution_id": demo.get("execution_id"),
        "work_dir": demo.get("work_dir"),
        "video_urls_count": len(demo.get("video_urls", [])),
        "final_path": demo.get("final_path"),
        "result": demo.get("result"),
    })
except Exception as e:
    print({"error": str(e)})


[FAL] using subscribe flow; route=fal-ai/veo3
[FAL] subscribe(0)
[FAL] subscribe(1)
[FAL] subscribe(2)
[FAL] fetched video URLs: ['https://v3b.fal.media/files/b/elephant/b5I8lLXwaGxDG536Lh7Ml_output.mp4', 'https://v3b.fal.media/files/b/penguin/qLqfvBhg_9nIhyHtqdG2i_output.mp4', 'https://v3b.fal.media/files/b/kangaroo/HphxNXOUiyVU1o3IbACA1_output.mp4']
{'execution_id': '55b073a7f7fd408f8cfbd4b7f9b00919', 'work_dir': '/tmp/n8n/55b073a7f7fd408f8cfbd4b7f9b00919', 'video_urls_count': 3, 'final_path': '/tmp/n8n/55b073a7f7fd408f8cfbd4b7f9b00919/final_output.mp4', 'result': {'videoUrl': 'http://localhost:3001/video/55b073a7f7fd408f8cfbd4b7f9b00919/final_output.mp4', 'executionId': '55b073a7f7fd408f8cfbd4b7f9b00919', 'message': 'Video processing complete'}}
